In [1]:
import pandas as pd
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv,dotenv_values
import json

c:\Users\belifakb\Downloads\UPM\GEN-AI\Development\AZURE-AI-VECTOR-SEARCH\openaiev\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# FILE_PATH = "C:/Users/belifakb/Downloads/UPM/GEN-AI/Development/AZURE-AI-VECTOR-SEARCH/azure_ai_vector_search/doc/CHAP04-BIOLOGY-CLASS11.pdf"
FILE_PATH = "C:/Users/belifakb/Downloads/UPM/GEN-AI/Development/AZURE-AI-VECTOR-SEARCH/azure_ai_vector_search/doc/Azure Common Data Platform - Operation Manual - CDP.pdf"

In [3]:
load_dotenv()
values_env = dotenv_values("../.env.sample")
MODEL_NAME = values_env['MODEL_NAME']

In [4]:
# Read the PDF file and return the text
def get_pdf_data(file_path, num_pages = 1):
    reader = PdfReader(file_path)
    full_doc_text = ""
    pages = reader.pages
    num_pages = len(pages) 
    
    try:
        for page in range(num_pages):
            current_page = reader.pages[page]
            text = current_page.extract_text()
            full_doc_text += text
    except:
        print("Error reading file")
    finally:
        return full_doc_text

In [5]:
# Divide the text into chunks of chunk_length 
# [ default is 500] characters
def get_chunks(fulltext:str,chunk_length =500) -> list:
    text = fulltext

    chunks = []
    while len(text) > chunk_length:
        last_period_index = text[:chunk_length].rfind('.')
        if last_period_index == -1:
            last_period_index = chunk_length
        chunks.append(text[:last_period_index])
        text = text[last_period_index+1:]
    chunks.append(text)

    return chunks

In [6]:
filename = FILE_PATH

In [7]:
FILE_PATH.split('/')[-1]

'Azure Common Data Platform - Operation Manual - CDP.pdf'

In [9]:
full_doc_text = get_pdf_data(filename)

In [10]:
print(f'Full doc text length: {len(full_doc_text)}')

Full doc text length: 43344


In [11]:
Lines =get_chunks(full_doc_text,500)

In [12]:
len(Lines)

100

In [13]:
type(Lines)

list

In [14]:
model = SentenceTransformer(MODEL_NAME)

c:\Users\belifakb\Downloads\UPM\GEN-AI\Development\AZURE-AI-VECTOR-SEARCH\openaiev\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
embeddings_all = model.encode(Lines,show_progress_bar=True)

Batches: 100%|██████████| 4/4 [00:37<00:00,  9.30s/it]


In [23]:
embeddings_all

array([[-0.01924709,  0.00871187, -0.05926949, ..., -0.01094172,
         0.03990278,  0.03409977],
       [-0.05356609, -0.02808592, -0.12278798, ...,  0.03618822,
         0.01923978,  0.00342018],
       [ 0.05533018, -0.05195809, -0.0556058 , ...,  0.02364424,
         0.01728379, -0.00147125],
       ...,
       [ 0.00904329, -0.05196362, -0.03948028, ...,  0.06055843,
         0.02239281,  0.05182135],
       [ 0.01384353,  0.00204655, -0.0285248 , ..., -0.05467969,
         0.04292374,  0.03390015],
       [-0.01819994,  0.01908312, -0.04957126, ...,  0.00423687,
        -0.00912765, -0.05189893]], dtype=float32)

In [16]:
len(embeddings_all)

100

In [17]:
embeddings_all[4].tolist()[:5]

[-0.028308985754847527,
 -0.022493818774819374,
 -0.04300372302532196,
 -0.08936355262994766,
 -0.0077858902513980865]

In [18]:
counter = 0

In [19]:
input_data = []

In [20]:
for line in Lines:
    d = {}
    d['id'] = str(counter)
    d['line'] = line
    d['embedding'] = embeddings_all[counter].tolist()
    d['filename'] = FILE_PATH.split('/')[-1]
    counter = counter + 1
    input_data.append(d)

In [24]:
input_data

[{'id': '0',
  'line': '   \n   \nDeliverIT – CDP (Common Data Platform)   V1.1A  \n  Confidential  \nTieto   2023-Oct-01 \n \n   \n  page 1/25   \n   \n© 2019  \nTieto Corporation   \nDeliverIT  \n \nService Operational Manual  \nUPM – CDP (Common Data Platform)  \n \n \n \n \n  \n \n \n \n \n \n \n \n \n \n \n \n     \n   \nDeliverIT – CDP (Common Data Platform)   V1.1A  \n  Confidential  \nTieto   2023-Oct-01 \n \n   \n  page 2/25   \n   \n© 2019  \nTieto Corporation   \nDeliverIT  \n \n  \nContents  \n1 About this document ................................ ...',
  'embedding': [-0.019247092306613922,
   0.00871187075972557,
   -0.0592694915831089,
   -0.04737403616309166,
   0.0020823844242841005,
   -0.0677763819694519,
   -0.03362158313393593,
   0.03964937850832939,
   -0.038719695061445236,
   0.047659192234277725,
   0.061930522322654724,
   0.031456585973501205,
   -0.006055186036974192,
   -0.044004786759614944,
   -0.01282401941716671,
   0.0419660285115242,
   0.06220170110

In [22]:
# Output embeddings to docVectors.json file
with open("../output/docVectors.json", "w") as f:
    json.dump(input_data, f)